In [1]:
import pandas as pd
import requests
import regex as re
from datetime import datetime as dt
from bs4 import BeautifulSoup

covidlink = "https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data"
poplink = "https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)#List"

covid_text = requests.get(covidlink).text
pop_text = requests.get(poplink).text
covidsoup = BeautifulSoup(covid_text,'html.parser')
popsoup = BeautifulSoup(pop_text,'html.parser')

covid_table = 'wikitable plainrowheaders sortable'
pop_table = 'nowrap sortable mw-datatable wikitable'
covid_table = covidsoup.find('table',{'class':covid_table}).tbody
pop_table = popsoup.find('table',{'class':pop_table}).tbody

covidrows = covid_table.find_all('tr')
covidcols = [v.text.replace('\n','').split('[')[0] for v in covidrows[0].find_all('th')][:4]
covidcols[0] = 'Location'
http://localhost:8889/notebooks/covid-19v2.ipynb#
def parseint(text):
    try:
        return int(re.sub('[^0-9]','',text))
    except ValueError:
        return 0

df = pd.DataFrame(columns=covidcols)
for i in range(2,len(covidrows)):
    try:
        tds = covidrows[i].find_all('th')[1].find_all('a')[0]
        tdr = covidrows[i].find_all('td')
        if len(tdr)==4:
            values = [tds.text.split(' (')[0].replace('&','and'), parseint(tdr[0].text), 
                      parseint(tdr[1].text), parseint(tdr[2].text)]
            df = df.append(pd.Series(values, index = covidcols), ignore_index=True)
    except IndexError:pass
vals = ["World"]
vals.extend(df.sum()[1:].values.tolist())
df = df.append(pd.Series(vals, index = covidcols), ignore_index=True)

poprows = pop_table.find_all('tr')
popcols = ['Location', 'Population-2019']

dfpop = pd.DataFrame(columns=popcols)
for i in range(1,len(poprows)):
    try:
        tds = poprows[i].find_all('td')
        values = [tds[0].find_all('a')[0].text,parseint(tds[4].text)]
    except IndexError:
        tds = poprows[i].find_all('th')
        values = [tds[0].find_all('a')[0].text,parseint(tds[4].text)]
    finally:
        dfpop = dfpop.append(pd.Series(values,index = popcols), ignore_index=True)

dfm = df.merge(dfpop,on = 'Location')
dfm = dfm.sort_values('Cases', ascending=False).reset_index(drop=True)
dfm.to_csv('covid-19-'+dt.today().strftime("%m-%d-%Y-%H-%M-%S")+'.csv',index=False)